In [10]:
from google.colab import drive 
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras import optimizers
from keras import layers
from keras import models

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import backend as K


base_dir = r'/content/gdrive/My Drive/MVP Datensatz/'
train_dir = r'/content/gdrive/My Drive/MVP Datensatz/Trainingsdatensatz5'


In [0]:
## create base_model

from keras.applications import Xception
base_model = Xception(
  weights = 'imagenet',
  include_top = False,
  pooling = 'avg', 
  input_shape = (299,299,3) # default vaules 
 )

In [7]:
## create train and validation generators

batch_s = 10  # global batch_size

## Train und Vaild Splitting

# Beispiel für mögliche Data Augmentations
train_datagen = ImageDataGenerator(featurewise_center = False,
                          samplewise_center = False,
                          featurewise_std_normalization = False,
                          samplewise_std_normalization = False,
                          rotation_range = 359,
                          width_shift_range = 30.0,
                          height_shift_range = 0.0,
                          brightness_range = None,
                          channel_shift_range = 50.0,
                          horizontal_flip = True, 
                          vertical_flip = True, 
                          rescale = 1./255,
                          validation_split = 0.2, 
                          interpolation_order = 1, 
                          dtype = 'float32')


valid_datagen = ImageDataGenerator(
                          rescale=1./255,
                          validation_split=0.2)

train_generator = valid_datagen.flow_from_directory(
    train_dir, 
    target_size = (299,299),
    batch_size = batch_s,
    class_mode = 'categorical',
    subset = 'training'
    )

validation_generator = valid_datagen.flow_from_directory(
    train_dir, 
    target_size = (299,299),
    batch_size = batch_s,
    class_mode = 'categorical',
    subset = 'validation'
)           

Found 80 images belonging to 5 classes.
Found 20 images belonging to 5 classes.


In [0]:
## extract features from data set

batch_size = batch_s
def extract_features(directory, sample_count, gen):
  features = np.zeros(shape=(sample_count, 2048)) 
  labels = np.zeros(shape=(sample_count, 5))
  generator = gen
  i=0
  for inputs_batch, labels_batch in generator:
    features_batch = base_model.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break
  return features, labels

train_features, train_labels = extract_features(train_dir, 80, train_generator)
validation_features, validation_labels = extract_features(train_dir, 20, validation_generator)


In [0]:
## create model

model = models.Sequential()
model.add(base_model)
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5,activation = 'softmax'))


In [12]:
# first: train only the top layers (which were randomly initialized)
# freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(
    train_generator,
    epochs = 10,
    validation_data = validation_generator
    )

Epoch 1/10
8/8 [==============================] - 6s 766ms/step - loss: 1.5645 - acc: 0.3000 - val_loss: 1.3090 - val_acc: 0.3000
Epoch 2/10
8/8 [==============================] - 2s 309ms/step - loss: 1.2498 - acc: 0.5625 - val_loss: 1.3526 - val_acc: 0.6000
Epoch 3/10
8/8 [==============================] - 2s 310ms/step - loss: 1.1350 - acc: 0.6000 - val_loss: 1.2747 - val_acc: 0.7500
Epoch 4/10
8/8 [==============================] - 2s 311ms/step - loss: 0.9216 - acc: 0.8375 - val_loss: 0.8822 - val_acc: 0.6500
Epoch 5/10
8/8 [==============================] - 2s 310ms/step - loss: 0.8859 - acc: 0.7125 - val_loss: 1.0592 - val_acc: 0.6500
Epoch 6/10
8/8 [==============================] - 2s 310ms/step - loss: 0.8345 - acc: 0.8125 - val_loss: 1.1460 - val_acc: 0.7000
Epoch 7/10
8/8 [==============================] - 2s 311ms/step - loss: 0.7252 - acc: 0.8125 - val_loss: 1.0589 - val_acc: 0.7000
Epoch 8/10
8/8 [==============================] - 2s 311ms/step - loss: 0.7493 - acc: 0.83

In [0]:
model.save("/content/gdrive/My Drive/Keras Models/Xception_5-1.h5")


In [15]:
## Validdierung auf Abfragebildern
testbild = base_dir + 'Testbilder/P1040695.JPG'
testbild = image.load_img(testbild, target_size=(299,299))
testbild = image.img_to_array(testbild)
testbild = np.expand_dims(testbild, axis=0)
predictions = model.predict(testbild)

predictions

array([[0., 0., 0., 1., 0.]], dtype=float32)

In [17]:
# image folder
folder_path = base_dir + 'Testbilder/'
# path to model
model_path = '/path/to/saved/model.h5'
# dimensions of images
img_width, img_height = 299, 299


# load all images into a list
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size = batch_s)
print(classes)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [0]:
def Vorhersage(Artikelfoto, Speicherort_KI_Modell, Name_KI_Modell):

    model = models.load_model(Speicherort_KI_Modell + Name_KI_Modell)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    img_width, img_height = 299, 299
    Artikelfoto = image.load_img(Artikelfoto, target_size=(img_width, img_height))
    Artikelfoto = image.img_to_array(Artikelfoto)
    Artikelfoto = np.expand_dims(Artikelfoto, axis=0)
    prediction = model.predict_proba(Artikelfoto)

    return {
        'category': np.argmax(prediction),
        'probs': {c: round(float(prediction[i]),5) for (i,c) in enumerate(prediction)}
    }


In [39]:
Vorhersage(base_dir + 'Testbilder/P1040695.JPG', '/content/gdrive/My Drive/Keras Models', '/Xception_5-1.h5')

TypeError: ignored